In [2]:
import numpy as np              # Массивы (матрицы, векторы, линейная алгебра)
import matplotlib.pyplot as plt # Научная графика
%matplotlib inline 
    # Говорим jupyter'у, чтобы весь графический вывод был в браузере, а не в отдельном окне
import pandas as pd             # Таблицы и временные ряды (dataframe, series)
import seaborn as sns           # Еще больше красивой графики для визуализации данных
import sklearn                  # Алгоритмы машинного обучения

# Удовлетворенность пассажиров авиакомпании

Предположим, что вы хотите оценить как зависит довольны ли пассажиры полетом на самолете в зависимости от разных факторов.

## Загружаем данные

Датасет изначально был разделен на тестовую и обучающую выборку. Для чистоты эксперимента не будем пользоваться этим разделением. Объединим обе таблицы и случайным образом перемешаем строки.

In [4]:
data_test = pd.read_csv("./datasets/test.csv")
data_train = pd.read_csv("./datasets/train.csv")
data = pd.concat([data_test, data_train])

Размеры таблицы:

In [ ]:
data.shape

In [ ]:
data.head()

Мы видим, что столбцы (признаки) имеют имена

- `Date` - Дата наблюдения
- `Location` - Общее название места расположения метеостанции
- `MinTemp` - Минимальная температура в градусах Цельсия в этот день
- `MaxTemp` - Максимальная температура в градусах Цельсия в этот день
- `Rainfall` - Количество осадков, выпавших за сутки в мм.
- `Evaporation` - Так называемое испарение из эвапорометра класса А (мм) за 24 часа до 9 утра. (*Эвапорометр - устройство измерения общего испарения при одновременном воздействии разных текущих климатических факторов, таких как: температура, влажность, атмосферные осадки, солнечная радиация, ветер*)
- `Sunshine` - Количество часов яркого солнечного света в сутки.
- `WindGustDir` - Направление самого сильного порыва ветра за 24 часа до полуночи
- `WindGustSpeed` - Скорость (км/ч) самого сильного порыва ветра за 24 часа до полуночи
- `WindDir9am` - Направление ветра в 9 утра
- `WindDir3pm` - Направление ветра в 15:00
- `WindSpeed9am` - Средняя скорость ветра (км/ч) за 10 минут до 9:00
- `WindSpeed3pm` - Средняя скорость ветра (км/ч) за 10 минут до 15:00
- `Humidity9am` - Влажность (в процентах) в 9 утра
- `Humidity3pm` - Влажность (в процентах) в 15:00
- `Pressure9am` - Атмосферное давление (гПа) на уровне среднего уровня моря в 9:00.
- `Pressure3pm` - Атмосферное давление (гПа) на уровне среднего уровня моря в 15:00.
- `Cloud9am` - Площадь неба, закрытая облаками в 9 утра. Измеряется в «октах» (часть от восьми). Показатель фиксирует, сколько восьмых неба закрыто облаками. Значение 0 указывает на абсолютно ясное небо, а значение 8 указывает на полную облачность.
- `Cloud3pm` - Площадь неба, закрытая облаками в 15:00
- `Temp9am` - Температура в градусах Цельсия в 9 утра
- `Temp3pm` - Температура в градусах Цельсия в 15:00
- `RainToday` - Булево значение: 1, если осадки (мм) за 24 часа до 9:00 превышают 1 мм, в противном случае 0
- `RainTomorrow` - Количество осадков на следующий день в мм. Используется для создания переменной ответа RainTomorrow. Своего рода мера «риска».

Признаки `RainToday` и `RainTomorrow` - бинарный, признаки `Location`, `WindGustDir`, `WindDir9am`, `WindDir3pm` - номинальные (категориальные), `Date` имеет тип дата, остальные признаки - количественные (числовые).

Требуется предсказать бинарный признак `RainTomorrow` по остальным признакам. Это *задача классификации*.

## Готовим данные

Приведем столбец `Date` к числовому типу

In [ ]:
from datetime import datetime, timedelta
data['Date'] = pd.to_datetime(data['Date'])
data['Date'] = data['Date'].sub(datetime(2008, 12, 1), axis=0).dt.days
data['Date']

In [ ]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
print(categorical_columns)
print(numerical_columns)

## Боремся с выбросами (outliers)

In [ ]:
rows_to_drop = []
for column in data.columns:
    if (column in categorical_columns):
        continue
    rows_to_drop += data[(data[column] < data[column].quantile(0.005))
    | (data[column] > data[column].quantile(0.995))].index.tolist()
data = data.drop(rows_to_drop)
data.shape

Для всех количественных признаков удаление выбросов привело к сокращению количества строк на 145460 - 136606 = 8854 строк

## Визуализация и описательная статистика

Матрица корреляции

In [ ]:
corr_mat = data.corr()
sns.heatmap(corr_mat, square=True, cmap='coolwarm')
pass

### Интерпретация
- Как видим минимальная температура `MinTemp` достаточно сильно коррелирует с максимальной MaxTemp, что очевидно, так как в жарком дне в среднем и минимальная и максимальная температура будут выше, а в холодном - ниже
- Также `MinTemp` и `MaxTemp` коррелируют с Temp9am и Temp3pm, что так же очевидно
- `Rainfall` слабо коререлирует с какими либо другими признаками. Наибольшая корреляция с `Humidity9am` и `Humidity3pm` (что очевидно - чем больше осадков, тем сильнее влажность), и с `Cloud9am` и `Cloud3pm` (чем облачнее, тем больше вероятность, что пойдет дождь)
- `Evaporation` (количество испарения из эвапорометра) коррелирует со всеми признаками связанными с температурой - чем выше температура, тем интенсивнее жидкость испаряется
- `Sunshine` так же коррелирует с признаками связанными с температурой - чем больше солнца, тем выше температура
- Признаки скорости ветра коррелируют только между собой, что очевидно так как ни от температуры, ни от давления, ни от других признаков они не зависят
- Коррелируют Humidity и Cloud - признаки влажности и облачности, что также связано с выпадением осадков
- И корреляция признаков, показывающих давление с другими признаками так же слабая

In [ ]:
data.describe()

##  Обрабатываем пропущенные значения

В данных присутствуют пропущенные значения:

In [ ]:
data.isna().sum()

Заполним медианами пропущенные значения в столбцах, соответствующих числовым признакам:

In [ ]:
data.fillna(data.median(axis = 0), axis=0 , inplace=True)

In [ ]:
data.isna().sum()

Пропущенные значения в столбцах, соответствующих категориальным признакам, заполним модой:

In [ ]:
for c in categorical_columns:
    data[c].fillna(data[c].mode().iloc[0], inplace=True)

In [ ]:
data.isna().sum()

## Векторизация

Обработка категориальных признаков

In [ ]:
data_describe = data.describe(include = [object])
binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
print(binary_columns, nonbinary_columns)

In [ ]:
data['RainToday'].unique()

In [ ]:
data.at[data['RainToday'] == 'No', 'RainToday'] = 0
data.at[data['RainToday'] == 'Yes', 'RainToday'] = 1
data['RainToday'].describe()

К небинарными признакам применим метод _векторизации_, 
который заключается в следующем.

Признак `j`, принимающий `s` значений, заменим на `s` признаков, принимащих значения `0` или `1`,
в зависимости от того, чему равно значение исходного признака `j`.

In [ ]:
data_nonbinary = pd.get_dummies(data[nonbinary_columns])
print(data_nonbinary.columns)

## Нормализация количественных признаков

Многие алгоритмы машинного обучения чувствительны к масштабированию данных.
К таким алгоритмам, например, относится метод ближайших соседей, машина опорных векторов и др.

В этом случае количественные признаки полезно _нормализовать_.
Это можно делать разными способами.
Например, каждый количественный признак приведем к нулевому среднему и единичному среднеквадратичному отклонению:

In [ ]:
data_numerical = data[numerical_columns]
data_numerical.describe()

In [ ]:
data_numerical = (data_numerical - data_numerical.mean(axis = 0))/data_numerical.std(axis = 0)
data_numerical.describe()

## Соединяем все в одну таблицу

In [ ]:
data = pd.concat((data_numerical, data_nonbinary, data[binary_columns]), axis = 1)
print(data.shape)

## 7. Разбиение данных на обучающую и тестовую выборки

Для предсказания будем использовать все признаки.

In [ ]:
X = data.drop('RainTomorrow', axis = 1)
y = data['RainTomorrow']

Разобьем данные на обучающую и тестовую выборки в пропорции 3:1 (75% - обучающая выборка, 25% - тестовая):

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 

N_train, N_test

## Классификатор ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)

In [ ]:
y_train_predict = knn.predict(X_train)
err_train  = np.mean(y_train  != y_train_predict)

In [ ]:
err_train

In [ ]:
y_test_predict = knn.predict(X_test)
err_test  = np.mean(y_test  != y_test_predict)

In [ ]:
err_test

Ошибка на обучающей выборке меньше из-за так называемого переобучения. На определенных данных алгоритм улавливает какие-то случайные закономерности, свойственные только конкретному набору данных, и используя их, ошибается меньше.

Матрица рассогласования:

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_test_predict))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(knn, X_test, y_test, cmap=plt.cm.Blues)
pass

Тру негативов больше чем тру позитивов, из-за несбалансированности классов. В `RainTomorrow` No составляет 78%, Yes - 22%

### Увеличим количество ближайших соседей до 9

In [ ]:
knn.set_params(n_neighbors=9)
knn.fit(X_train, y_train)

In [ ]:
y_train_predict = knn.predict(X_train)
err_train  = np.mean(y_train  != y_train_predict)
err_train

In [ ]:
y_test_predict = knn.predict(X_test)
err_test  = np.mean(y_test  != y_test_predict)
err_test

### Увеличим количество ближайших соседей до 17

In [ ]:
knn.set_params(n_neighbors=17)
knn.fit(X_train, y_train)

In [ ]:
y_train_predict = knn.predict(X_train)
err_train  = np.mean(y_train  != y_train_predict)
err_train

In [ ]:
y_test_predict = knn.predict(X_test)
err_test  = np.mean(y_test  != y_test_predict)
err_test

### Увеличим количество ближайших соседей до 51

In [ ]:
knn.set_params(n_neighbors=51)
knn.fit(X_train, y_train)

In [ ]:
y_train_predict = knn.predict(X_train)
err_train  = np.mean(y_train  != y_train_predict)
err_train

In [ ]:
y_test_predict = knn.predict(X_test)
err_test  = np.mean(y_test  != y_test_predict)
err_test

### Подбор параметров

In [ ]:
# Долго!
from sklearn.model_selection import GridSearchCV
nnb = [1, 3, 5, 9, 15, 19, 25, 35, 45, 55]
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid = {'n_neighbors': nnb}, cv=10)
grid.fit(X_train, y_train)

best_cv_err = 1 - grid.best_score_
best_n_neighbors = grid.best_estimator_.n_neighbors
print(best_cv_err, best_n_neighbors)

Наиболее эффективен метод с 19-ю ближайшими соседями, хотя вероятности для разного числа соседей на тестовой выборке примерно одинаковы.